### 전처리

In [ ]:
from google import colab
colab.drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
 !pip install catboost

     |████████████████████████████████| 76.3 MB 1.2 MB/s 


In [ ]:
import os
import sys
import platform
import random
import math
from typing import List ,Dict, Tuple

import pandas as pd
import numpy as np
 
import sklearn 
from sklearn.model_selection import StratifiedKFold , KFold
from sklearn.metrics import f1_score 

from catboost import Pool,CatBoostClassifier

print(f"- os: {platform.platform()}")
print(f"- python: {sys.version}")
print(f"- pandas: {pd.__version__}")
print(f"- numpy: {np.__version__}")
print(f"- sklearn: {sklearn.__version__}")

- os: Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic
- python: 3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]
- pandas: 1.1.5
- numpy: 1.19.5
- sklearn: 1.0.1


In [ ]:
#유진

train_data = pd.read_csv('/content/drive/MyDrive/Jobcare_data/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Jobcare_data/test.csv')
SEED = 43
code_d = pd.read_csv('/content/drive/MyDrive/Jobcare_data/속성_D_코드.csv').iloc[:,:-1]
code_h = pd.read_csv('/content/drive/MyDrive/Jobcare_data/속성_H_코드.csv')
code_l = pd.read_csv('/content/drive/MyDrive/Jobcare_data/속성_L_코드.csv')

train_data.shape , test_data.shape

((501951, 35), (46404, 34))

In [ ]:
DATA_PATH = '/content/drive/MyDrive/Jobcare_data/'
train_data = pd.read_csv(f'{DATA_PATH}train.csv')
test_data = pd.read_csv(f'{DATA_PATH}test.csv')
SEED = 42
code_d = pd.read_csv(f'{DATA_PATH}속성_D_코드.csv').iloc[:,:-1]
code_h = pd.read_csv(f'{DATA_PATH}속성_H_코드.csv')
code_l = pd.read_csv(f'{DATA_PATH}속성_L_코드.csv')

train_data.shape , test_data.shape

((501951, 35), (46404, 34))

In [ ]:
code_d.columns= ["attribute_d","attribute_d_d","attribute_d_s","attribute_d_m"]
code_h.columns= ["attribute_h","attribute_h_m","attribute_h_l"]
code_l.columns= ["attribute_l","attribute_l_d","attribute_l_s","attribute_l_m","attribute_l_l"]

In [ ]:
def merge_codes(df:pd.DataFrame,df_code:pd.DataFrame,col:str)->pd.DataFrame:#매개변수의 타입이 data.frame
#print(test.__annotations__) 참고로 import annotation해서 함수안 파라미터를 확인할 수 있다.
    df = df.copy()
    df_code = df_code.copy()
    df_code = df_code.add_prefix(f"{col}_")
    df_code.columns.values[0] = col
    return pd.merge(df,df_code,how="left",on=col)

In [ ]:
def preprocess_data(
                    df:pd.DataFrame,
                    is_train:bool = True,
                    cols_merge:List[Tuple[str,pd.DataFrame]] = []  ,
                    cols_equi:List[Tuple[str,str]]= [] ,
                    cols_drop:List[str] = ["id","person_prefer_f","person_prefer_g" ,"contents_open_dt"]
                    )->Tuple[pd.DataFrame,np.ndarray]:
    df = df.copy()

    y_data = None
    if is_train:
        y_data = df["target"].to_numpy()
        df = df.drop(columns="target")

    for col, df_code in cols_merge:
        df = merge_codes(df,df_code,col)

    cols = df.select_dtypes(bool).columns.tolist()
    df[cols] = df[cols].astype(int)

    for col1, col2 in cols_equi:
        df[f"{col1}_{col2}"] = (df[col1] == df[col2] ).astype(int)

    df = df.drop(columns=cols_drop)
    return (df , y_data)

In [ ]:
# 소분류 중분류 대분류 속성코드 merge 컬럼명 및 데이터 프레임 리스트
cols_merge = [
              ("person_prefer_d_1" , code_d),
              ("person_prefer_d_2" , code_d),
              ("person_prefer_d_3" , code_d),
              ("contents_attribute_d" , code_d),
              ("person_prefer_h_1" , code_h),
              ("person_prefer_h_2" , code_h),
              ("person_prefer_h_3" , code_h),
              ("contents_attribute_h" , code_h),
              ("contents_attribute_l" , code_l),
]

# 회원 속성과 콘텐츠 속성의 동일한 코드 여부에 대한 컬럼명 리스트
cols_equi = [

    ("contents_attribute_c","person_prefer_c"),
    ("contents_attribute_e","person_prefer_e"),

    ("person_prefer_d_2_attribute_d_s" , "contents_attribute_d_attribute_d_s"),
    ("person_prefer_d_2_attribute_d_m" , "contents_attribute_d_attribute_d_m"),
    ("person_prefer_d_2_attribute_d_d" , "contents_attribute_d_attribute_d_d"),
    ("person_prefer_d_3_attribute_d_s" , "contents_attribute_d_attribute_d_s"),
    ("person_prefer_d_3_attribute_d_m" , "contents_attribute_d_attribute_d_m"),
    ("person_prefer_d_3_attribute_d_d" , "contents_attribute_d_attribute_d_d"),

    ("person_prefer_h_1_attribute_h_m" , "contents_attribute_h_attribute_h_m"),#attribute_h_m ,l,none
    ("person_prefer_h_1_attribute_h_l" , "contents_attribute_h_attribute_h_l"),
    ("person_prefer_h_2_attribute_h_m" , "contents_attribute_h_attribute_h_m"),
    ("person_prefer_h_3_attribute_h_m" , "contents_attribute_h_attribute_h_m"),
    ("person_prefer_h_2_attribute_h_l" , "contents_attribute_h_attribute_h_l"),
    ("person_prefer_h_3_attribute_h_l" , "contents_attribute_h_attribute_h_l"),

]

# 학습에 필요없는 컬럼 리스트
cols_drop = ["id","person_prefer_f","person_prefer_g" ,
             "person_prefer_d_3_attribute_d_m_contents_attribute_d_attribute_d_m", "person_prefer_h_3_attribute_h_l"]

In [ ]:
x_train, y_train = preprocess_data(train_data, cols_merge = cols_merge , cols_equi= cols_equi , cols_drop = cols_drop)
x_test, _ = preprocess_data(test_data,is_train = False, cols_merge = cols_merge , cols_equi= cols_equi  , cols_drop = cols_drop)
x_train.shape , y_train.shape , x_test.shape #row수

((501951, 67), (501951,), (46404, 67))

In [ ]:
cat_features = x_train.columns[x_train.nunique() > 2].tolist()

##K-fold

In [ ]:
is_holdout = False
#기존 train과 val데이터 분할 안함
n_splits = 5
#데이터 분할 수 k-fold 검정
#iterations = 3000
#반복수
#patience = 50
#최소 인내수
cv = KFold(n_splits=n_splits, shuffle=True, random_state=SEED)

optuna돌리면 나오는 최적 파라미터

In [ ]:
#유진 best params ver3

best_params ={'iterations': 1422, 
              'objective': 'CrossEntropy',  
              'bootstrap_type': 'Bayesian','od_wait': 666, 'learning_rate': 0.9782109291187356, 'reg_lambda': 70.72533306533951, 
              'random_strength': 47.81900485462368, 'depth': 3, 'min_data_in_leaf': 20, 'leaf_estimation_iterations': 5, 'one_hot_max_size': 1, 
              'bagging_temperature': 0.07799233624102353,
               "cat_features": cat_features,
               "eval_metric":'F1',
               'task_type' : 'GPU'
               }

k-fold 검증을통한 모델 학습(fit)

In [ ]:
(x_test.shape[0])

46404

In [ ]:
#유진 ver_3

scores = []
models = []


for tri, vai in cv.split(x_train):
    preds = []
    model = CatBoostClassifier(**best_params)
    model.fit(x_train.iloc[tri], y_train[tri],
            eval_set=[(x_train.iloc[vai], y_train[vai])],
            early_stopping_rounds = 400,
            verbose=100
        )
    models.append(model)
    scores.append(model.get_best_score()["validation"]["F1"])
    if is_holdout:
        break    

0:	learn: 0.6237145	test: 0.6253576	best: 0.6253576 (0)	total: 218ms	remaining: 5m 9s
100:	learn: 0.6677331	test: 0.6654330	best: 0.6713464 (69)	total: 17.4s	remaining: 3m 47s
200:	learn: 0.6717121	test: 0.6667002	best: 0.6713464 (69)	total: 34.7s	remaining: 3m 30s
300:	learn: 0.6747106	test: 0.6693515	best: 0.6713464 (69)	total: 52.2s	remaining: 3m 14s
400:	learn: 0.6760429	test: 0.6687782	best: 0.6713464 (69)	total: 1m 9s	remaining: 2m 57s
bestTest = 0.6713463957
bestIteration = 69
Shrink model to first 70 iterations.
0:	learn: 0.4741875	test: 0.4716871	best: 0.4716871 (0)	total: 229ms	remaining: 5m 25s
100:	learn: 0.6671883	test: 0.6706090	best: 0.6714614 (86)	total: 17.3s	remaining: 3m 46s
200:	learn: 0.6729880	test: 0.6714356	best: 0.6742442 (162)	total: 34.7s	remaining: 3m 30s
300:	learn: 0.6752176	test: 0.6724017	best: 0.6744925 (268)	total: 52.2s	remaining: 3m 14s
400:	learn: 0.6769340	test: 0.6723062	best: 0.6744925 (268)	total: 1m 9s	remaining: 2m 57s
500:	learn: 0.6783944	te

예측값들 val

In [ ]:
print(scores)
print(np.mean(scores))

[0.6713463956516142, 0.6744924872122763, 0.674767663030088, 0.6725314787846366, 0.6705158200202833]
0.6727307689397797


In [ ]:
def thresholds_search(y_true,y_prob):
  thresholds = np.linspace(0,1,101)
  f1_score_arr = np.array([f1_score(y_true,(y_prob>t).astype(np.int)) for t in thresholds])
  best_score = np.max(f1_score_arr)
  best_th = thresholds[np.atgmax(f1_score_arr)]

  return best_score,best_th

test데이터에 target proba

In [ ]:
threshold = 0.33791

pred_list = []
scores = []
for i,(tri, vai) in enumerate( cv.split(x_train) ):
    thresholds_search
    pred = models[i].predict_proba(x_train.iloc[vai])[:, 1]
    pred = np.where(pred >= threshold , 1, 0)
    score = f1_score(y_train[vai],pred)
    scores.append(score)
    pred = models[i].predict_proba(x_test)[:, 1]
    pred_list.append(pred) 
print(scores)
print(np.mean(scores))

pred = np.mean( pred_list , axis = 0 )
pred = np.where(pred >= threshold , 1, 0)

[0.7145623299727956, 0.7184590638048971, 0.7146109570152795, 0.7153172405977302, 0.7147904274675871]
0.715548003771658


In [ ]:
pred = np.mean( pred_list , axis = 0 )
pred = np.where(pred >= threshold , 1, 0)

In [ ]:
#유진 submission 저장

sample_submission = pd.read_csv(f'{DATA_PATH}sample_submission.csv')
sample_submission['target'] = pred
sample_submission
sample_submission.to_csv('/content/drive/MyDrive/submission_3.csv', index=False)